In [1]:
import sys
import os
from oepul_chat.data_download import get_all_oepul_pdf_anchor_tags, download_pdfs, download_html
from oepul_chat.readers.custom_html_reader import CustomHTMLReader
import requests
from llama_index.schema import Document
from bs4 import BeautifulSoup

# Autoreload local library
%load_ext autoreload
%autoreload 2


# Scrape oepul regulatory text from the web

There is a [website](https://www.ama.at/formulare-merkblaetter#18053) which has a dump of all PDF files regarding the oepul regulations. This notebook is used to download all these PDFs into the data folder.

In [2]:
links = get_all_oepul_pdf_anchor_tags()
download_pdfs(links)

In [3]:
# Manual other links: 
links = ["https://www.ama.at/getattachment/4d988952-847f-4847-a650-cf2e866d2ebe/20230131_Merkblatt_MFA2023_V3.pdf", 
         "https://www.ama.at/getattachment/abb20701-bbad-46cc-a5ef-deb2ab03ebc6/Merkblatt_Flaechenmonitoring_2023.pdf",
         "https://www.ama.at/getattachment/05e60ad0-cbcb-40d3-820d-315e7fb49979/Nutzungsarten_Codes_Varianten_2023_V4.pdf"]

# Scrape guides from Bio Austria

In [4]:
download_html(url='https://www.bio-austria.at/a/bauern/aktueller-planungsstand-zu-bio-im-oepul-2023/',
              folder_path="data/BIO_Austria/", filename="aktueller-planungsstand-zu-bio-im-oepul-2023")
docs = CustomHTMLReader().load_data(file="data/BIO_Austria/aktueller-planungsstand-zu-bio-im-oepul-2023.html")

In [9]:
# Function to convert HTML article element to structured dictionary
def html_to_dict(element):
    result = {}
    for child in element.children:
        if child.name is not None:
            if child.name in ['h1', 'h2', 'h3']:
                result[child.name] = child.get_text().strip()
            elif child.name == 'p':
                if 'text' not in result:
                    result['text'] = []
                result['text'].append(child.get_text().strip())
            elif child.name in ['ul', 'ol']:
                if 'lists' not in result:
                    result['lists'] = []
                current_list = []
                for li in child.find_all('li'):
                    current_list.append(li.get_text().strip())
                result['lists'].append({child.name: current_list})
            elif child.name == 'div':
                if 'divs' not in result:
                    result['divs'] = []
                result['divs'].append(html_to_dict(child))
    return result


# Function to convert HTML article element to a list of llama index Documents
def html_to_list_of_dicts_with_tags(element, header_path=[]):
    result = []
    for child in element.children:
        if child.name is not None:
            tag_type = child.name
            if tag_type in ['h1', 'h2', 'h3']:
                header_path.append(child.get_text().strip())
            elif tag_type == 'p':

                result.append(Document(
                    text=child.get_text().strip(),
                    metadata={
                        'File Name': filename,
                        "Content Type": "text",
                        'Header Path': '/'.join(header_path),
                        'tag': tag_type
                    }
                ))
                # result.append({'header_path': '/'.join(header_path),
                #               'tag': tag_type, 'content': child.get_text().strip()})
            elif tag_type in ['ul', 'ol']:
                current_list = []
                for li in child.find_all('li'):
                    current_list.append(li.get_text().strip())
                result.append(Document(
                    text=str(current_list),
                    metadata={
                        'File Name': filename,
                        "Content Type": "text",
                        'Header Path': '/'.join(header_path),
                        'tag': tag_type
                    }
                ))
                # result.append({'header_path': '/'.join(header_path),
                #               'tag': tag_type, 'content': current_list})
            elif tag_type == 'div':
                result.extend(html_to_list_of_dicts_with_tags(
                    child, header_path.copy()))
    return result



filename = "test"
file = "data/BIO_Austria/aktueller-planungsstand-zu-bio-im-oepul-2023.html"
html = open(file, 'r').read()
soup = BeautifulSoup(html, 'html.parser')
# Finding the specific div with id 'cs-inhalte'
article_element = soup.find('article')



html_elements = None
try:
    html_elements = html_to_list_of_dicts_with_tags(article_element)
except Exception as e:
    print("Failed to convert HTML to dictionary.")

[e.text for e in html_elements]

['Stand 21.10.2022',
 'Am 13.09.2022 wurde der österreichische GAP – Strategieplan, in dem die Ausgestaltung der nächsten Förderperiode festgelegt ist, von der Europäischen Kommission genehmigt. Das entsprechende Dokument wurde vom Landwirtschaftsministerium hier veröffentlicht:',
 'info.bml.gv.atTeil des GAP – Strategieplans ist auch das österreichische Agrarumweltprogramm (ÖPUL). Wir haben die wichtigsten Punkte, die es im ÖPUL ab 2023 zu beachten gibt auf dieser Seite zusammengefasst – angefangen von der Bio-Maßnahme bis hin zu den kombinierbaren Maßnahmen.',
 'Den gesamten Entwurf für das ÖPUL 2023 mit allen Details finden Sie hier:info.bmlrt.gv.at',
 'Zusätzlich zu den ÖPUL – Auflagen sind in der Förderperiode ab 2023 auch die Vorgaben der Konditionalität zu beachten. In diesem Zusammenhang weisen wir darauf hin, dass bei der Anlage von Biodiversitätsflächen auf Ackerflächen im Rahmen der ÖPUL – Biomaßnahme auch die Vorgaben der GLÖZ 8 zu berücksichtigen sind. In der Bio Austria –